In [1]:
import numpy as np
import pandas as pd
import scanpy as sc


Bad key "text.kerning_factor" on line 4 in
/home/debnatho/anaconda3/envs/scanpy_scgen/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
import anndata 
import celltypist
from celltypist import models

In [3]:
#Load the early normalized data (H5AD) file from 2021 submission:
adata_train = sc.read_h5ad("/data/analysis/preeclampsia_2019/analysis/images/Final_umap_nature/Placenta_normalized_Seurat_markers.h5ad")
adata_train.obs['tissue_time']

index
VL_AAACCTGAGATCCCGC-1_1      Villi_early
VL_AAACCTGAGGACAGAA-1_1      Villi_early
VL_AAACCTGAGGCTAGGT-1_1      Villi_early
VL_AAACCTGCAAGCGAGT-1_1      Villi_early
VL_AAACCTGCAATCACAC-1_1      Villi_early
                                ...     
DC_TTTGGTTTCATCGGAT-1_22    Decidua_late
DC_TTTGGTTTCTGATTCT-1_22    Decidua_late
DC_TTTGTCAAGTTGTAGA-1_22    Decidua_late
DC_TTTGTCACAGCTGTGC-1_22    Decidua_late
DC_TTTGTCACATTATCTC-1_22    Decidua_late
Name: tissue_time, Length: 101031, dtype: category
Categories (4, object): ['Decidua_early', 'Decidua_late', 'Villi_early', 'Villi_late']

In [4]:
#Build a custom model on the control decidua (early & late term):  
subset= ['Decidua_early', 'Decidua_late']
adata_train= adata_train[adata_train.obs['tissue_time'].isin(subset)]
adata_train.obs['cell_type_semifinal_v2']

index
DC_AAACCTGAGAAACGAG-1_4             dTcell
DC_AAACCTGAGAAGCCCA-1_4               dEpC
DC_AAACCTGAGACCTAGG-1_4             dTcell
DC_AAACCTGAGACTAAGT-1_4     dMAC_classical
DC_AAACCTGAGCACGCCT-1_4     dMAC_classical
                                 ...      
DC_TTTGGTTTCATCGGAT-1_22            dTcell
DC_TTTGGTTTCTGATTCT-1_22    dMAC_activated
DC_TTTGTCAAGTTGTAGA-1_22              dSMC
DC_TTTGTCACAGCTGTGC-1_22             DSC_2
DC_TTTGTCACATTATCTC-1_22    dMAC_activated
Name: cell_type_semifinal_v2, Length: 40725, dtype: category
Categories (22, object): ['DSC_1', 'DSC_2', 'dDC', 'dEVT', ..., 'dSCT', 'dSMC', 'dTcell', 'dVEC']

In [5]:
adata_train.obs['cell_type_semifinal_v2'].cat.categories #categories in decidua 

Index(['DSC_1', 'DSC_2', 'dDC', 'dEVT', 'dEpC', 'dFB_1', 'dFB_2',
       'dGranulocyte', 'dLEC', 'dLEC_dysfunctional', 'dMAC_activated',
       'dMAC_classical', 'dMSC', 'dMonocyte', 'dNK_1', 'dNK_2', 'dNK_prol',
       'dPlasmaCell', 'dSCT', 'dSMC', 'dTcell', 'dVEC'],
      dtype='object')

In [6]:
#Filter control cells only:
adata_train= adata_train[adata_train.obs['disease']== 'C']
adata_train

View of AnnData object with n_obs × n_vars = 27750 × 26115
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'stim', 'percent.mt', 'S.Score', 'G2M.Score', 'Phase', 'old.ident', 'integrated_snn_res.0.5', 'seurat_clusters', 'group', 'disease', 'time', 'tissue', 'tissue_time', 'integrated_snn_res.1', 'integrated_snn_res.2', 'integrated_snn_res.3', 'cell_type', 'RNA_snn_res.0.8', 'Merged_cell_type_PC35', 'cell_type_merged', 'sub_cluster', 'sub_cluster_final', 'cell_type_semifinal', 'cell_type_semifinal_v2'
    uns: 'cell_type_semifinal_colors', 'umap_density_disease_params', 'umap_density_group_params', 'umap_density_params', 'umap_density_tissue_params'
    obsm: 'X_umap'

In [7]:
#Subset cell types/states of interest:
subset= ['DSC_1', 'DSC_2', 'dDC', 'dEVT', 'dEpC', 'dFB_1', 'dFB_2','dGranulocyte', 'dLEC', 'dLEC_dysfunctional', 
         'dMAC_activated','dMAC_classical', 'dMSC', 'dMonocyte', 'dNK_1', 'dNK_2', 'dNK_prol',
       'dPlasmaCell', 'dSCT', 'dSMC', 'dTcell', 'dVEC']

#Training dataset for decidua: 
adata_train_dec= adata_train[adata_train.obs['cell_type_semifinal_v2'].isin(subset)]
adata_train_dec.obs['cell_type_semifinal_v2'].value_counts()

dEpC                  6656
dMAC_classical        4483
dNK_1                 4417
dTcell                2849
dMAC_activated        1570
dNK_2                 1444
dLEC                  1313
dVEC                  1184
DSC_1                  783
dSMC                   552
dPlasmaCell            529
dMSC                   476
dNK_prol               361
dGranulocyte           285
dFB_1                  185
dMonocyte              165
DSC_2                  148
dFB_2                  146
dDC                    137
dEVT                    31
dLEC_dysfunctional      26
dSCT                    10
Name: cell_type_semifinal_v2, dtype: int64

In [8]:
from celltypist.models import Model

import logging
logging.basicConfig(level=logging.INFO, format="%(message)s")
logger = logging.getLogger(__name__)

set_level = logger.setLevel
info = logger.info
warn = logger.warning
error = logger.error
debug = logger.debug

In [9]:
import numpy as np
import pandas as pd
import scanpy as sc
from anndata import AnnData
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from typing import Optional, Union
from celltypist.models import Model
from . import logger
from scipy.sparse import spmatrix
from datetime import datetime

def _to_vector(_vector_or_file):
    """
    For internal use. Turn a file into an array.
    """
    if isinstance(_vector_or_file, str):
        try:
            return pd.read_csv(_vector_or_file, header=None)[0].values
        except Exception as e:
            raise Exception(f"🛑 {e}")
    else:
        return _vector_or_file

def _to_array(_array_like) -> np.ndarray:
    """
    For internal use. Turn an array-like object into an array.
    """
    if isinstance(_array_like, pd.DataFrame):
        return _array_like.values
    elif isinstance(_array_like, spmatrix):
        return _array_like.toarray()
    elif isinstance(_array_like, np.matrix):
        return np.array(_array_like)
    elif isinstance(_array_like, np.ndarray):
        return _array_like
    else:
        raise ValueError(f"🛑 Please provide a valid array-like object as input")

def _prepare_data(X, labels, genes, transpose) -> tuple:
    """
    For internal use. Prepare data for celltypist training.
    """
    if (X is None) or (labels is None):
        raise Exception("🛑 Missing training data and/or training labels. Please provide both arguments")
    if isinstance(X, AnnData) or (isinstance(X, str) and X.endswith('.h5ad')):
        adata = sc.read(X) if isinstance(X, str) else X
        adata.var_names_make_unique()
        if adata.X.min() < 0:
            logger.info("👀 Detected scaled expression in the input data, will try the .raw attribute")
            try:
                indata = adata.raw.X.copy()
                genes = adata.raw.var_names.copy()
            except Exception as e:
                raise Exception(f"🛑 Fail to use the .raw attribute in the input object. {e}")
        else:
            indata = adata.X.copy()
            genes = adata.var_names.copy()
        if isinstance(labels, str) and (labels in adata.obs):
            labels = adata.obs[labels]
        else:
            labels = _to_vector(labels)
    elif isinstance(X, str) and X.endswith(('.csv', '.txt', '.tsv', '.tab', '.mtx', '.mtx.gz')):
        adata = sc.read(X)
        if transpose:
            adata = adata.transpose()
        if X.endswith(('.mtx', '.mtx.gz')):
            if genes is None:
                raise Exception("🛑 Missing `genes`. Please provide this argument together with the input mtx file")
            genes = _to_vector(genes)
            if len(genes) != adata.n_vars:
                raise ValueError(f"🛑 The number of genes provided does not match the number of genes in {X}")
            adata.var_names = np.array(genes)
        adata.var_names_make_unique()
        if not float(adata.X.max()).is_integer():
            logger.warn(f"⚠️ Warning: the input file seems not a raw count matrix. The trained model may be biased")
        sc.pp.normalize_total(adata, target_sum=1e4)
        sc.pp.log1p(adata)
        indata = adata.X.copy()
        genes = adata.var_names.copy()
        labels = _to_vector(labels)
    elif isinstance(X, str):
        raise ValueError("🛑 Invalid input. Supported types: .csv, .txt, .tsv, .tab, .mtx, .mtx.gz and .h5ad")
    else:
        logger.info("👀 The input training data is processed as an array-like object")
        indata = X.copy()
        if transpose:
            indata = indata.transpose()
        if isinstance(indata, pd.DataFrame):
            genes = indata.columns.copy()
        else:
            if genes is None:
                raise Exception("🛑 Missing `genes`. Please provide this argument together with the input training data")
            genes = _to_vector(genes)
        labels = _to_vector(labels)
    return indata, labels, genes

def _LRClassifier(indata, labels, C, solver, max_iter, n_jobs, **kwargs) -> LogisticRegression:
    """
    For internal use. Get the logistic Classifier.
    """
    no_cells = len(labels)
    if solver is None:
        solver = 'sag' if no_cells>50000 else 'lbfgs'
    elif solver not in ('liblinear', 'lbfgs', 'newton-cg', 'sag', 'saga'):
        raise ValueError(f"🛑 Invalid `solver`, should be one of `liblinear`, `lbfgs`, `newton-cg`, `sag`, and `saga`")
    logger.info(f"🏋️ Training data using logistic regression")
    if no_cells > 100000:
        logger.warn(f"⚠️ Warning: it may take a long time to train this dataset with {no_cells} cells, try to decrease `max_iter` if the training does not finish in practical time")
    classifier = LogisticRegression(C = C, solver = solver, max_iter = max_iter, multi_class = 'ovr', n_jobs = n_jobs, **kwargs)
    classifier.fit(indata, labels)
    return classifier

def _SGDClassifier(indata, labels,
                   alpha, max_iter, n_jobs,
                   mini_batch, batch_number, batch_size, epochs, balance_cell_type, **kwargs) -> SGDClassifier:
    """
    For internal use. Get the SGDClassifier.
    """
    classifier = SGDClassifier(loss = 'log', alpha = alpha, max_iter = max_iter, n_jobs = n_jobs, **kwargs)
    if not mini_batch:
        logger.info(f"🏋️ Training data using SGD logistic regression")
        if len(labels) > 100000:
            logger.warn(f"⚠️ Warning: it may take a long time to train this dataset with {len(labels)} cells, try to decrease `max_iter` if the training does not finish in practical time")
        classifier.fit(indata, labels)
    else:
        logger.info(f"🏋️ Training data using mini-batch SGD logistic regression")
        no_cells = len(labels)
        if no_cells < 10000:
            logger.warn(f"⚠️ Warning: the number of cells ({no_cells}) is not big enough to conduct a proper mini-batch training. You may consider using traditional SGD classifier (mini_batch = False)")
        if no_cells <= batch_size:
            raise ValueError(f"🛑 Number of cells ({no_cells}) is fewer than the batch size ({batch_size}). Decrease `batch_size`, or use SGD directly (mini_batch = False)")
        no_cells_sample = min([batch_number*batch_size, no_cells])
        starts = np.arange(0, no_cells_sample, batch_size)
        if balance_cell_type:
            celltype_freq = np.unique(labels, return_counts = True)
            len_celltype = len(celltype_freq[0])
            mapping = pd.Series(1 / (celltype_freq[1]*len_celltype), index = celltype_freq[0])
            p = mapping[labels].values
        for epoch in range(1, (epochs+1)):
            logger.info(f"⏳ Epochs: [{epoch}/{epochs}]")
            if not balance_cell_type:
                sampled_cell_index = np.random.choice(no_cells, no_cells_sample, replace = False)
            else:
                sampled_cell_index = np.random.choice(no_cells, no_cells_sample, replace = False, p = p)
            for start in starts:
                classifier.partial_fit(indata[sampled_cell_index[start:start+batch_size]], labels[sampled_cell_index[start:start+batch_size]], classes = np.unique(labels))
    return classifier

def train(X = None,
          labels: Optional[Union[str, list, tuple, np.ndarray, pd.Series, pd.Index]] = None,
          genes: Optional[Union[str, list, tuple, np.ndarray, pd.Series, pd.Index]] = None,
          transpose_input: bool = False,
          check_expression: bool = True,
          #LR param
          C: float = 1.0, solver: Optional[str] = None, max_iter: int = 1000, n_jobs: Optional[int] = None,
          #SGD param
          use_SGD: bool = False, alpha: float = 0.0001,
          #mini-batch
          mini_batch: bool = False, batch_number: int = 100, batch_size: int = 1000, epochs: int = 10, balance_cell_type: bool = False,
          #feature selection
          feature_selection: bool = False, top_genes: int = 500,
          #description
          date: str = '', details: str = '', url: str = '',
          #other SGD param
          **kwargs
         ) -> Model:
    """
    Train a celltypist model using mini-batch (optional) logistic classifier with a global solver or stochastic gradient descent (SGD) learning.
    Parameters
    ----------
    X
        Path to the input count matrix (supported types are csv, txt, tsv, tab and mtx) or AnnData (h5ad).
        Also accepts the input as an :class:`~anndata.AnnData` object, or any array-like objects already loaded in memory.
        See `check_expression` for detailed format requirements.
        A cell-by-gene format is desirable (see `transpose_input` for more information).
    labels
        Path to the file containing cell type label per line corresponding to the cells in `X`.
        Also accepts any list-like objects already loaded in memory (such as an array).
        If `X` is specified as an AnnData, this argument can also be set as a column name from cell metadata.
    genes
        Path to the file containing one gene per line corresponding to the genes in `X`.
        Also accepts any list-like objects already loaded in memory (such as an array).
        Note `genes` will be extracted from `X` where possible (e.g., `X` is an AnnData or data frame).
    transpose_input
        Whether to transpose the input matrix. Set to `True` if `X` is provided in a gene-by-cell format.
        (Default: `False`)
    check_expression
        Check whether the expression matrix in the input data is supplied as required.
        Except the case where a path to the raw count table file is specified, all other inputs for `X` should be in log1p normalized expression to 10000 counts per cell.
        Set to `False` if you want to train the data regardless of the expression formats.
        (Default: `True`)
    C
        Inverse of L2 regularization strength for traditional logistic classifier. A smaller value can possibly improve model generalization while at the cost of decreased accuracy.
        This argument is ignored if SGD learning is enabled (`use_SGD = True`).
        (Default: 1.0)
    solver
        Algorithm to use in the optimization problem for traditional logistic classifier.
        The default behavior is to choose the solver according to the size of the input data.
        This argument is ignored if SGD learning is enabled (`use_SGD = True`).
    max_iter
        Maximum number of iterations before reaching the minimum of the cost function.
        Try to decrease `max_iter` if the cost function does not converge for a long time.
        This argument is for both traditional and SGD logistic classifiers, and will be ignored if mini-batch SGD training is conducted (`use_SGD = True` and `mini_batch = True`).
        (Default: 1000)
    n_jobs
        Number of CPUs used. Default to one CPU. `-1` means all CPUs are used.
        This argument is for both traditional and SGD logistic classifiers.
    use_SGD
        Whether to implement SGD learning for the logistic classifier.
        (Default: `False`)
    alpha
        L2 regularization strength for SGD logistic classifier. A larger value can possibly improve model generalization while at the cost of decreased accuracy.
        This argument is ignored if SGD learning is disabled (`use_SGD = False`).
        (Default: 0.0001)
    mini_batch
        Whether to implement mini-batch training for the SGD logistic classifier.
        Setting to `True` may improve the training efficiency for large datasets (for example, >100k cells).
        This argument is ignored if SGD learning is disabled (`use_SGD = False`).
        (Default: `False`)
    batch_number
        The number of batches used for training in each epoch. Each batch contains `batch_size` cells.
        For datasets which cannot be binned into `batch_number` batches, all batches will be used.
        This argument is relevant only if mini-batch SGD training is conducted (`use_SGD = True` and `mini_batch = True`).
        (Default: 100)
    batch_size
        The number of cells within each batch.
        This argument is relevant only if mini-batch SGD training is conducted (`use_SGD = True` and `mini_batch = True`).
        (Default: 1000)
    epochs
        The number of epochs for the mini-batch training procedure.
        The default values of `batch_number`, `batch_size`, and `epochs` together allow observing ~10^6 training cells.
        This argument is relevant only if mini-batch SGD training is conducted (`use_SGD = True` and `mini_batch = True`).
        (Default: 10)
    balance_cell_type
        Whether to balance the cell type frequencies in mini-batches during each epoch.
        Setting to `True` will sample rare cell types with a higher probability, ensuring close-to-even cell type distributions in mini-batches.
        This argument is relevant only if mini-batch SGD training is conducted (`use_SGD = True` and `mini_batch = True`).
        (Default: `False`)
    feature_selection
        Whether to perform two-pass data training where the first round is used for selecting important features/genes.
        If `True`, the training time will be approximately doubled.
        (Default: `False`)
    top_genes
        The number of top genes selected from each class/cell-type based on their absolute regression coefficients.
        The final feature set is combined across all classes (i.e., union).
        (Default: 500)
    date
        Free text of the date of the model. Default to the time when the training is completed.
    details
        Free text of the description of the model.
    url
        Free text of the (possible) download url of the model.
    **kwargs
        Other keyword arguments passed to :class:`~sklearn.linear_model.LogisticRegression` (`use_SGD = False`) or :class:`~sklearn.linear_model.SGDClassifier` (`use_SGD = True`).
    Returns
    ----------
    :class:`~celltypist.models.Model`
        An instance of the :class:`~celltypist.models.Model` trained by celltypist.
    """
    #prepare
    logger.info("🍳 Preparing data before training")
    indata, labels, genes = _prepare_data(X, labels, genes, transpose_input)
    indata = _to_array(indata)
    labels = np.array(labels)
    genes = np.array(genes)
    #check
    if check_expression and (np.abs(np.expm1(indata[0]).sum()-10000) > 1):
        raise ValueError("🛑 Invalid expression matrix, expect log1p normalized expression to 10000 counts per cell")
    if len(labels) != indata.shape[0]:
        raise ValueError(f"🛑 Length of training labels ({len(labels)}) does not match the number of input cells ({indata.shape[0]})")
    if len(genes) != indata.shape[1]:
        raise ValueError(f"🛑 The number of genes ({len(genes)}) provided does not match the number of genes in the training data ({indata.shape[1]})")
    #filter
    flag = indata.sum(axis = 0) == 0
    if flag.sum() > 0:
        logger.info(f"✂️ {flag.sum()} non-expressed genes are filtered out")
        indata = indata[:, ~flag]
        genes = genes[~flag]
    #scaler
    logger.info(f"⚖️ Scaling input data")
    scaler = StandardScaler()
    indata = scaler.fit_transform(indata)
    indata = np.clip(indata, a_min = None, a_max = 10)
    #classifier
    if use_SGD:
        classifier = _SGDClassifier(indata = indata, labels = labels, alpha = alpha, max_iter = max_iter, n_jobs = n_jobs, mini_batch = mini_batch, batch_number = batch_number, batch_size = batch_size, epochs = epochs, balance_cell_type = balance_cell_type, **kwargs)
    else:
        classifier = _LRClassifier(indata = indata, labels = labels, C = C, solver = solver, max_iter = max_iter, n_jobs = n_jobs, **kwargs)
    #feature selection -> new classifier and scaler
    if feature_selection:
        logger.info(f"🔎 Selecting features")
        if len(genes) <= top_genes:
            raise ValueError(f"🛑 The number of genes ({len(genes)}) is fewer than the `top_genes` ({top_genes}). Unable to perform feature selection")
        gene_index = np.argpartition(np.abs(classifier.coef_), -top_genes, axis = 1)[:, -top_genes:]
        gene_index = np.unique(gene_index)
        logger.info(f"🧬 {len(gene_index)} features are selected")
        genes = genes[gene_index]
        indata = indata[:, gene_index]
        logger.info(f"🏋️ Starting the second round of training")
        if use_SGD:
            classifier = _SGDClassifier(indata = indata, labels = labels, alpha = alpha, max_iter = max_iter, n_jobs = n_jobs, mini_batch = mini_batch, batch_number = batch_number, batch_size = batch_size, epochs = epochs, balance_cell_type = balance_cell_type, **kwargs)
        else:
            classifier = _LRClassifier(indata = indata, labels = labels, C = C, solver = solver, max_iter = max_iter, n_jobs = n_jobs, **kwargs)
        scaler.mean_ = scaler.mean_[gene_index]
        scaler.var_ = scaler.var_[gene_index]
        scaler.scale_ = scaler.scale_[gene_index]
        scaler.n_features_in_ = len(gene_index)
    #model finalization
    classifier.features = genes
    if not date:
        date = str(datetime.now())
    description = {'date': date, 'details': details, 'url': url, 'number_celltypes': len(classifier.classes_)}
    logger.info(f"✅ Model training done!")
    #return Model(classifier, scaler, description)
    return Model(classifier, scaler)

In [10]:
def write(early_model, file: str) -> None:
        """Write out the model."""
        obj = dict(Model = early_model.classifier, Scaler_ = early_model.scaler)
        file = os.path.splitext(file)[0] + '.pkl'
        with open(file, 'wb') as output:
            pickle.dump(obj, output)

In [11]:
#Train model on the "adata_train_dec"
decidua_model = train(adata_train_dec, labels = 'cell_type_semifinal_v2') #done!! 

🍳 Preparing data before training
✂️ 1457 non-expressed genes are filtered out
⚖️ Scaling input data
🏋️ Training data using logistic regression
✅ Model training done!


In [12]:
import os
import pathlib
import json
import pickle
import requests
import numpy as np
import pandas as pd
from typing import Optional, Union
from scipy.special import expit
from . import logger

#Note that, this model serves as a template for supervised classification of samples bases on 2021 annotations. 
write(decidua_model, '/data/analysis/preeclampsia_2019/placenta_atlas_2022/placenta_celltypist_model/model_from_sp014_control_decidua.pkl')

In [13]:
#test data: latest Cellbdender filtered decidua object (SP136 samples were not sequenced at this point)
#Model: trained on decidua controls. 
#Logistic Regression classifier used as implemented in CellTypist. 
predictions = celltypist.annotate(filename= "/data/analysis/preeclampsia_2019/placenta_atlas_2022/decidua_anndata/Decidua_cellbender_normalized_matrix_270122.h5ad", 
                                  model = '/data/analysis/preeclampsia_2019/placenta_atlas_2022/placenta_celltypist_model/model_from_sp014_control_decidua.pkl', 
                                  majority_voting = True)

📁 Input file is '/data/analysis/preeclampsia_2019/placenta_atlas_2022/decidua_anndata/Decidua_cellbender_normalized_matrix_270122.h5ad'
⏳ Loading data...
🔬 Input data has 44232 cells and 26603 genes
🧙 Matching reference genes
🧩 24619 features used for prediction
🧙 Scaling input data
🖋️ Predicting labels
✅ Prediction done!
🧙 Over-clustering input data with resolution set to 20
/home/debnatho/anaconda3/envs/scanpy_scgen/lib/python3.6/site-packages/scanpy/utils.py:618: DeprecationWarning: Use is_view instead of isview, isview will be removed in the future.
  if adata.isview:
/home/debnatho/anaconda3/envs/scanpy_scgen/lib/python3.6/site-packages/scanpy/neighbors/__init__.py:89: DeprecationWarning: Use is_view instead of isview, isview will be removed in the future.
  if adata.isview:  # we shouldn't need this here...
/home/debnatho/anaconda3/envs/scanpy_scgen/lib/python3.6/site-packages/numba/typed_passes.py:293: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified 

In [14]:
predictions.predicted_labels

,predicted labels,over clustering,predicted labels after majority voting
ACGTCAACAAGGACAC-1-0,dEpC,252,dEpC
TTAGGACCACACAGAG-1-0,DSC_2,96,DSC_2
ATCATCTTCCTTTCGG-1-0,dEpC,76,dEpC
ATCTACTGTTCCTCCA-1-0,DSC_1,145,dMSC
TTATGCTTCATTGCCC-1-0,dEpC,139,dEpC
...,...,...,...
AACTGGTAGCGACGTA-1-8,dMAC_activated,47,dMAC_activated
AACTCAGAGCAACGGT-1-8,dMAC_activated,217,dMAC_activated
CTTGGCTCACCTGGTG-1-8,dMAC_activated,58,dMAC_activated
CAGTCCTAGAATTCCC-1-8,dMAC_activated,20,dMAC_activated


In [15]:
adata_test= sc.read_h5ad("/data/analysis/preeclampsia_2019/placenta_atlas_2022/decidua_anndata/Decidua_cellbender_normalized_matrix_270122.h5ad")

adata_annot= adata_test.copy()
adata_annot.obs= predictions.predicted_labels #Write the "predicted_labels" in the metadata. 
adata_annot.obs['predicted labels after majority voting'].value_counts()

dNK_1             8490
dMAC_classical    7716
dEpC              7203
dTcell            6112
dLEC              2981
dMAC_activated    2938
dNK_2             2051
dVEC              1774
DSC_1             1170
dSMC               860
dPlasmaCell        557
dMSC               476
dSCT               456
dGranulocyte       348
DSC_2              292
dFB_1              169
dNK_prol           161
dFB_2              160
dDC                160
dEVT               158
Name: predicted labels after majority voting, dtype: int64

In [16]:
#Write the predicted celltypist labels as a .obs class: 
adata_test.obs['majority_voting']= adata_annot.obs['predicted labels after majority voting']
adata_test.obs['predicted_labels']= adata_annot.obs['predicted labels']
adata_test.obs['majority_voting'].value_counts()

dNK_1             8490
dMAC_classical    7716
dEpC              7203
dTcell            6112
dLEC              2981
dMAC_activated    2938
dNK_2             2051
dVEC              1774
DSC_1             1170
dSMC               860
dPlasmaCell        557
dMSC               476
dSCT               456
dGranulocyte       348
DSC_2              292
dFB_1              169
dNK_prol           161
dFB_2              160
dDC                160
dEVT               158
Name: majority_voting, dtype: int64

In [17]:
results_annot= "/data/analysis/preeclampsia_2019/placenta_atlas_2022/placenta_celltypist_model/Decidua_atlas_celltypist_annotated_2701.h5ad" #should we replace it? 
adata_test.write(results_annot)

#If more PE samples are added, we've to either repeat this or use scANVI for label-transfer. 

... storing 'maternal_BMI' as categorical
... storing 'maternal_age' as categorical
... storing 'majority_voting' as categorical
... storing 'predicted_labels' as categorical
